# Train your own custom fine tuned chat bot with your own data

You need your own OpenAI API key which might cost you money.
You will also need an API key for Telegram and your user ID to limit the chatbot use to your bot.

In [ ]:
# Import necessary packages
import os
import telebot
from llama_index import GPTVectorStoreIndex, download_loader, StorageContext, load_index_from_storage, LLMPredictor,ServiceContext
from langchain import OpenAI,ChatOpenAI
from telegram import Update
from telegram.ext import filters, MessageHandler, ApplicationBuilder, CommandHandler, ContextTypes

#needed because of neverending loop
import nest_asyncio
nest_asyncio.apply()

os.environ['OPENAI_API_KEY'] = 'your_key_here'
Telegram_API_key = 'your telegram key here'
telegram_user_id = 12345 #Enter your telegram user id here to limit bot use to yourself

In [ ]:
#Create index
def create_index():
    SimpleDirectoryReader = download_loader("SimpleDirectoryReader")

    loader = SimpleDirectoryReader('./Data', recursive=True, exclude_hidden=True)
    documents = loader.load_data()
    
    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003",max_tokens=512)) #"gpt-3.5-turbo","text-davinci-003"

    # configure service context
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

    index = GPTVectorStoreIndex.from_documents(documents,service_context=service_context)
    query_engine = index.as_query_engine()
    #store index (not working at the moment)
    #index.storage_context.persist('./storage')
    return query_engine
query_engine = create_index()

In [ ]:
#test response
query_engine.query('Hallo World').response

In [ ]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    #Initialise your bot with a welcome message
    await context.bot.send_message(chat_id=update.effective_chat.id, text="I'm a bot, please talk to me!")

async def bot(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if update.message.from_user.id==telegram_user_id:
        #generate response from message text
        response = query_engine.query(update.message.text).response
        #chunk up response in case it's too big
        chunks = [response[i:i+4096] for i in range(0, len(response), 4096)]
        # Loop through the chunks and send them to the bot
        for chunk in chunks:
            await context.bot.send_message(chat_id=update.effective_chat.id, text=chunk)
    else:
        pass

def run_bot(query_engine):
    application = ApplicationBuilder().token(Telegram_API_key).build()

    start_handler = CommandHandler('start', start)
    llm_handler = MessageHandler(filters.TEXT & (~filters.COMMAND), bot)
    
    application.add_handler(start_handler)
    application.add_handler(llm_handler)

    application.run_polling()

In [ ]:
#run bot - go to telegram to talk to your bot
run_bot(query_engine)